In [1]:
import pulp
import numpy as np
import time

In [2]:
start = time.time()

In [3]:
# Create Random demand and Pobability for SGD

import random
l1=[60,80,100,120,140]
l2=[100,150,200,250,300]
l3=[90,120,150,180,210]
l4=[70,120,170,220,270]

w=[0.02275 , 0.13591, 0.68268, 0.13591, 0.02275]
# print(random.choices(l1, weights=w))
def rnddm(l, w):
    dm = random.choices(l, weights=w)
    p = w[l.index(dm[0])]
    return dm[0], p

In [4]:
demands = ['l1','l2','l3','l4']
h=1   # holding cost
p=4   # penalty cost for shortage 
c0i=0.5 #basic direct transshipment cost
cs=100000000000000000 #basic direct transshipment cost

In [5]:
#define trans from i to j
count=0
trans=[]
for i in demands:
    for j in demands:
        trans.append((i,j))
    
    #del trans[count][count]
    count=count+1
del trans[::5]

In [6]:
#define trans from j to i
count=0
trans_r=[]
for i in demands:
    for j in demands:
        trans_r.append((j,i))
    
    #del trans[count][count]
    count=count+1
del trans_r[::5]

In [7]:
#first-stage variable: order-up-to quantity
s = pulp.LpVariable.dicts("s",(demands) ,lowBound=0, cat='Continuous')
#ending inventory held at retailer i. 
e=pulp.LpVariable.dicts("e",(demands) ,lowBound=0, cat='Continuous')
#stock at retailer i used to satisfy demand at retailer i
f=pulp.LpVariable.dicts("f",(demands) ,lowBound=0, cat='Continuous') 
#inventory at retailer i increased through replenishment
q=pulp.LpVariable.dicts("q",(demands) ,lowBound=0, cat='Continuous')
#amount of shortage at the retailer i. (Assume all past shortage are omitted)
r=pulp.LpVariable.dicts("r",(demands) ,lowBound=0, cat='Continuous')
#stock at retailer i used to meet demand at retailer j, using the transshipment option.
t=pulp.LpVariable.dicts("t",(trans) ,lowBound=0, cat='Continuous')

# S coefficient
Cs=[[1,0,0,0],
   [0,0,0,0],
   [1,0,0,0],
   [0,1,0,0],
   [0,0,0,0],
   [0,1,0,0],
   [0,0,1,0],
   [0,0,0,0],
   [0,0,1,0],
   [0,0,0,1],
   [0,0,0,0],
   [0,0,0,1],
   [0,0,0,0]]


Cs = [[-1,0,0,0],[0,0,0,0],[-1,0,0,0],[0,-1,0,0],[0,0,0,0],[0,-1,0,0],[0,0,-1,0],[0,0,0,0],
    [0,0,-1,0],[0,0,0,-1],[0,0,0,0],[0,0,0,-1],[0,0,0,0]]


In [8]:
def SGD(sample_size, iteration_time, alpha): 
    Ds = []
    Ps = []
    
    for i in range(sample_size):
        l1_sample = rnddm(l1,w)
        l2_sample = rnddm(l2,w)
        l3_sample = rnddm(l3,w)
        l4_sample = rnddm(l4,w)

        demand = {'l1':l1_sample[0],'l2':l2_sample[0],'l3':l3_sample[0],'l4':l4_sample[0]}
        probability = l1_sample[1]*l2_sample[1]*l3_sample[1]*l4_sample[1]

        Ds.append(demand)
        Ps.append(probability)
    
    scenarios = list(range(0,len(Ds)))
    subgradient = [0,0,0,0] #initial subgradient = -c, in this case, is 0
    Cs = [[-1,0,0,0],
           [0,0,0,0],
           [-1,0,0,0],
           [0,-1,0,0],
           [0,0,0,0],
           [0,-1,0,0],
           [0,0,-1,0],
           [0,0,0,0],
           [0,0,-1,0],
           [0,0,0,-1],
           [0,0,0,0],
           [0,0,0,-1],
           [0,0,0,0]]




    s = {'l1': 50, 'l2': 200, 'l3': 50, 'l4': 180} # initialize s
    
    
    for i in range(iteration_time):
        step=i
        up=0
        sub=[]
        for j in scenarios:
            transship = pulp.LpProblem("transship",pulp.LpMinimize)
            # Objective
            transship += pulp.lpSum([h * e[i] for i in demands]
                                    + [cs * t[(i,j)] for (i,j) in trans[3:]]
                                    + [c0i * t[(i,j)] for (i,j) in trans[:3]]
                                    + [p * r[i] for i in demands])

            opt=[]
            obj=[]
            d=Ds[j]
            start=0
            up=0
            for i in demands:  

                transship += e[i] + \
                                f[i] + \
                                pulp.lpSum(t[j] for j in trans[start:start+3])==s[i]

                transship += r[i] + \
                                f[i] + \
                                pulp.lpSum(t[j] for j in trans_r[start:start+3])==d[i]

                transship += e[i]+q[i]==s[i]
                start=start+3

            transship += pulp.lpSum(r[i]+q[i] for i in demands)==pulp.lpSum(d[i] for i in demands)

            #solve    
            transship.solve()
            
            
            
            dual = []
            for name, c in list(transship.constraints.items()):
                dual.append(c.pi)

          
            subgradient = subgradient - Ps[j]*(np.array(dual).T.dot(Cs))
            
#             up=up+Ps[j]*transship.objective.value()
            
            sub.append(Ps[j]*transship.objective.value())
            
        subgradient=subgradient/np.array(Ps).sum()
        
        up=np.array(sub).sum()/np.array(Ps).sum()
        
        if (abs(subgradient[0]-0)<0.01)&(abs(subgradient[1]-0)<0.01)&(abs(subgradient[2]-0)<0.01)&(abs(subgradient[3]-0)<0.01):
            break
        
        else:
            s_new = list(s.values()) - (alpha)*np.array(subgradient) 

            for n in range(4):
                if s_new[n]<0:
                    s_new[n] = 0

            s = {'l1':s_new[0],'l2':s_new[1],'l3':s_new[2],'l4':s_new[3]} # update s to next iteration
        print(subgradient,s)

In [9]:
SGD(sample_size=50,iteration_time=3000,alpha=0.1)

[-4.         -0.46359699 -4.          0.60963183] {'l1': 50.4, 'l2': 200.0463596990328, 'l3': 50.4, 'l4': 179.93903681684827}
[-5.22909424  0.6084302  -5.22909424  0.79695558] {'l1': 50.922909424346685, 'l2': 199.98551667867085, 'l3': 50.922909424346685, 'l4': 179.85934125932775}
[-5.60676241 -3.63828284 -5.60676241  0.85451521] {'l1': 51.48358566517645, 'l2': 200.34934496259527, 'l3': 51.48358566517645, 'l4': 179.77388973843142}
[-5.72280985 -0.36706682 -5.72280985  0.87220176] {'l1': 52.05586665018957, 'l2': 200.3860516447763, 'l3': 52.05586665018957, 'l4': 179.6866695621724}
[-5.75846816  0.63809137 -5.75846816  0.87763637] {'l1': 52.63171346627086, 'l2': 200.3222425075532, 'l3': 52.63171346627086, 'l4': 179.59890592488213}
[-5.76942502  0.94694991 -5.76942502  0.87930628] {'l1': 53.20865596796192, 'l2': 200.22754751657837, 'l3': 53.20865596796192, 'l4': 179.5109752963856}
[-5.77279177  1.04185397 -5.77279177  0.8798194 ] {'l1': 53.78593514487187, 'l2': 200.12336211934084, 'l3': 53.

[-5.75805057  0.95107714 -5.77428516  0.88004701] {'l1': 81.49957236171626, 'l2': 200.20113502038942, 'l3': 81.5024376880027, 'l4': 175.19877780375057}
[-5.75687803  1.04312216 -5.77428516  0.88004701] {'l1': 82.0752601651546, 'l2': 200.09682280440194, 'l3': 82.07986620437413, 'l4': 175.1107731027491}
[-5.75651775  1.07140516 -5.77428516  0.88004701] {'l1': 82.65091193972849, 'l2': 199.9896822881891, 'l3': 82.65729472074555, 'l4': 175.02276840174764}
[-5.75640704 -3.49602288 -5.77428516  0.88004701] {'l1': 83.2265526435849, 'l2': 200.3392845757042, 'l3': 83.23472323711698, 'l4': 174.93476370074617}
[-5.75637302 -0.3233541  -5.77428516  0.88004701] {'l1': 83.80218994570252, 'l2': 200.37161998531167, 'l3': 83.81215175348841, 'l4': 174.8467589997447}
[-5.75636257  0.65152314 -5.77428516  0.88004701] {'l1': 84.37782620255577, 'l2': 200.30646767159936, 'l3': 84.38958026985983, 'l4': 174.75875429874324}
[-5.75635936  0.95107714 -5.77428516  0.88004701] {'l1': 84.9534621382269, 'l2': 200.2113

[-4.93225637  0.6592391  -5.64055378  0.88004701] {'l1': 111.06991098949185, 'l2': 200.25893036917225, 'l3': 112.559766743904, 'l4': 170.4465239496714}
[-4.93017636  0.95344805 -5.63847377  0.88004701] {'l1': 111.56292862540901, 'l2': 200.16358556435756, 'l3': 113.1236141207616, 'l4': 170.35851924866992}
[-4.92953723  1.04385068 -5.63783464  0.88004701] {'l1': 112.05588234810752, 'l2': 200.05920049650754, 'l3': 113.68739758440078, 'l4': 170.27051454766845}
[-4.92934084  1.07162902 -5.63763825  0.88004701] {'l1': 112.54881643196373, 'l2': 199.9520375948435, 'l3': 114.25116140919776, 'l4': 170.182509846667}
[-4.92928049 -3.41425038 -5.6375779   0.88004701] {'l1': 113.04174448132294, 'l2': 200.29346263283747, 'l3': 114.81491919949777, 'l4': 170.09450514566552}
[-4.92926195 -0.29822757 -5.63755936  0.88004701] {'l1': 113.53467067644077, 'l2': 200.32328538986647, 'l3': 115.37867513555639, 'l4': 170.00650044466406}
[-4.92925625  0.65924385 -5.63755366  0.88004701] {'l1': 114.02759630179925, 

[ 0.23136052 -0.34214029 -0.42642707 -0.347011  ] {'l1': 123.26224194890402, 'l2': 199.9835078889033, 'l3': 128.37116342680812, 'l4': 170.0254932049737}
[ 0.23136052 -0.41348882 -0.42642707  0.50300452] {'l1': 123.23910589731187, 'l2': 200.02485677069555, 'l3': 128.41380613343168, 'l4': 169.97519275251122}
[ 0.23136052  0.62382712 -0.42642707 -0.14721556] {'l1': 123.21596984571973, 'l2': 199.96247405867726, 'l3': 128.45644884005523, 'l4': 169.98991430876814}
[ 0.23136052 -0.11667257 -0.42642707 -0.347011  ] {'l1': 123.19283379412758, 'l2': 199.97414131593288, 'l3': 128.49909154667878, 'l4': 170.02461540917898}
[ 0.23136052 -0.34420855 -0.42642707  0.50300452] {'l1': 123.16969774253543, 'l2': 200.00856217092937, 'l3': 128.54173425330234, 'l4': 169.97431495671648}
[ 0.23136052  0.64511511 -0.42642707 -0.14721556] {'l1': 123.14656169094329, 'l2': 199.94405065944775, 'l3': 128.5843769599259, 'l4': 169.9890365129734}
[ 0.23136052 -0.11013133 -0.42642707 -0.347011  ] {'l1': 123.1234256393511

[ 0.23136052 -0.10994156 -0.42642707 -0.347011  ] {'l1': 122.0128951629281, 'l2': 199.95100696719535, 'l3': 130.67386958448, 'l4': 170.00969288066847}
[ 0.23136052 -0.34214029 -0.42642707  0.50300452] {'l1': 121.98975911133596, 'l2': 199.9852209960516, 'l3': 130.71651229110356, 'l4': 169.95939242820597}
[ 0.23136052 -0.41348882 -0.42642707 -0.14721556] {'l1': 121.96662305974381, 'l2': 200.02656987784385, 'l3': 130.7591549977271, 'l4': 169.9741139844629}
[ 0.23136052  0.62382712 -0.42642707 -0.347011  ] {'l1': 121.94348700815166, 'l2': 199.96418716582556, 'l3': 130.80179770435066, 'l4': 170.00881508487373}
[ 0.23136052 -0.11667257 -0.42642707  0.50300452] {'l1': 121.92035095655952, 'l2': 199.9758544230812, 'l3': 130.84444041097422, 'l4': 169.95851463241124}
[ 0.23136052 -0.34420855 -0.42642707 -0.14721556] {'l1': 121.89721490496737, 'l2': 200.01027527807767, 'l3': 130.88708311759777, 'l4': 169.97323618866815}
[ 0.23136052  0.64511511 -0.42642707 -0.347011  ] {'l1': 121.87407885337522, '

...

[0.20971296 1.07063967 1.07934936 0.86766485] {'l1': 100.40636653358038, 'l2': 200.01841505597764, 'l3': 149.98665586353403, 'l4': 170.1039490731353}
[0.2164772  1.07986057 0.03625822 0.8762423 ] {'l1': 100.38471881404307, 'l2': 199.91042899938302, 'l3': 149.9830300416331, 'l4': 170.016324843308}
[ 0.21855567  0.02345445 -0.28425611  0.87887792] {'l1': 100.36286324733543, 'l2': 199.90808355456323, 'l3': 150.01145565274086, 'l4': 169.92843705103905}
[ 0.21919433 -0.30115122  0.66924386 -0.03171961] {'l1': 100.34094381468763, 'l2': 199.93819867669563, 'l3': 149.94453126678184, 'l4': 169.93160901155014}
[ 0.21939057 -0.40089396 -0.08975636 -0.31152215] {'l1': 100.319004757725, 'l2': 199.9782880728232, 'l3': 149.9535069027137, 'l4': 169.9627612265496}
[ 0.21945087 -0.43154227 -0.32297706 -0.39749807] {'l1': 100.29705967072927, 'l2': 200.02144229963832, 'l3': 149.98580460864187, 'l4': 170.00251103397963}
[ 0.2194694   0.61827977 -0.39463961  0.48749118] {'l1': 100.2751127308638, 'l2': 199.9

[ 0.21947755 -0.11018079 -0.39501027 -0.14735386] {'l1': 100.21859931346299, 'l2': 199.9500527222988, 'l3': 150.0318485544996, 'l4': 169.9789276358788}
[ 0.2194776  -0.3422138   0.63521204 -0.3470535 ] {'l1': 100.19665155391739, 'l2': 199.98427410184314, 'l3': 149.96832735089873, 'l4': 170.0136329858081}
[ 0.21947761 -0.4135114  -0.10021344  0.50299147] {'l1': 100.174703792848, 'l2': 200.0256252423075, 'l3': 149.9783486947921, 'l4': 169.96333383911028}
[ 0.21947762  0.62382018 -0.32619024 -0.14721957] {'l1': 100.15275603131039, 'l2': 199.96324322431943, 'l3': 150.0109677191292, 'l4': 169.97805579659416}
[ 0.21947762 -0.11667471  0.65635861 -0.34701224] {'l1': 100.1308082696289, 'l2': 199.97491069483218, 'l3': 149.94533185818534, 'l4': 170.01275702029145}
[ 0.21947762 -0.34420921 -0.09371566  0.50300415] {'l1': 100.1088605079032, 'l2': 200.00933161535693, 'l3': 149.9547034237869, 'l4': 169.96245660571162}
[ 0.21947762  0.64511491 -0.32419365 -0.14721568] {'l1': 100.08691274616393, 'l2':

[0.12363146 0.95346199 0.96174741 0.75439792] {'l1': 100.45990174961516, 'l2': 200.16189516551526, 'l3': 150.18079088225696, 'l4': 170.2497460767036}
[0.19002663 1.04385496 1.0521078  0.84143837] {'l1': 100.44089908677816, 'l2': 200.05750966916747, 'l3': 150.07558010230812, 'l4': 170.1656022400356}
[0.21042811 1.07163033 1.07987316 0.86818359] {'l1': 100.41985627606152, 'l2': 199.95034663583738, 'l3': 149.96759278660775, 'l4': 170.07878388056594}
[0.21669694 0.02092552 0.03641917 0.8764017 ] {'l1': 100.3981865818412, 'l2': 199.94825408430677, 'l3': 149.96395086989838, 'l4': 169.99114371095135}
[ 0.21862319 -0.3019283  -0.28420666 -0.03248048] {'l1': 100.37632426289447, 'l2': 199.97844691386067, 'l3': 149.99237153549706, 'l4': 169.9943917593572}
[ 0.21921507 -0.40113274 -0.38272649 -0.31175595] {'l1': 100.35440275546213, 'l2': 200.01856018742433, 'l3': 150.0306441849321, 'l4': 170.02556735415502}
[0.21939694 0.62762382 0.63898651 0.51383747] {'l1': 100.33246306097307, 'l2': 199.95579780

[ 0.21947688 -0.11663869 -0.39501344 -0.14711901] {'l1': 100.25150357183404, 'l2': 199.97379614241316, 'l3': 150.0234867225181, 'l4': 169.97374184273514}
[ 0.21947739 -0.34419814  0.63521106 -0.34698134] {'l1': 100.22955583265356, 'l2': 200.0082159564151, 'l3': 149.959965616448, 'l4': 170.0084399762369}
[ 0.21947755  0.64511831 -0.10021374  0.50301364] {'l1': 100.20760807784183, 'l2': 199.9437041250634, 'l3': 149.96998699030996, 'l4': 169.95813861212167}
[ 0.2194776  -0.11013035 -0.32619034 -0.14721276] {'l1': 100.18566031822702, 'l2': 199.9547171602399, 'l3': 150.00260602385563, 'l4': 169.97285988825183}
[ 0.21947761 -0.3421983   0.65635858 -0.34701014] {'l1': 100.16371255713636, 'l2': 199.98893699008855, 'l3': 149.93697016574131, 'l4': 170.00756090258713}
[ 0.21947762 -0.41350664 -0.09371566  0.50300479] {'l1': 100.14176479559222, 'l2': 200.0302876543724, 'l3': 149.94634173221232, 'l4': 169.9572604236759}
[ 0.21947762  0.62382164 -0.32419365 -0.14721548] {'l1': 100.11981703390873, 'l

[0.12363146 0.95346199 0.955251   0.74860162] {'l1': 100.5100687440576, 'l2': 200.16774033091852, 'l3': 150.18940373555134, 'l4': 170.26155546561307}
[0.19002663 1.04385496 1.05011162 0.83965732] {'l1': 100.49106608122077, 'l2': 200.06335483456314, 'l3': 150.0843925731701, 'l4': 170.17758973389493}
[0.21042811 1.07163033 1.07925978 0.86763633] {'l1': 100.47002327050417, 'l2': 199.95619180123074, 'l3': 149.97646659467054, 'l4': 170.09082610136738}
[0.21669694 0.02092552 0.03623069 0.87623353] {'l1': 100.44835357628386, 'l2': 199.95409924969942, 'l3': 149.97284352526128, 'l4': 170.0032027478952}
[ 0.21862319 -0.3019283  -0.28426457  0.87887523] {'l1': 100.42649125733713, 'l2': 199.9842920792531, 'l3': 150.00126998213696, 'l4': 169.91531522490698}
[ 0.21921507 -0.40113274  0.66924126 -0.03172043] {'l1': 100.4045697499048, 'l2': 200.0244053528167, 'l3': 149.93434585606005, 'l4': 169.91848726816093}
[ 0.21939694  0.62762382 -0.08975716 -0.3115224 ] {'l1': 100.38263005541573, 'l2': 199.96164

In [10]:
end = time.time()
print(end-start)

920.4312827587128
